In [511]:
import pandas as pd
import numpy as np
import csv
import gmaps
import requests
import json
import gmaps.datasets

from config import gkey
gmaps.configure(api_key=gkey)

In [512]:
# Import the population.csv file as a DataFrame
state_df = pd.read_csv("resources/state_populations_2000_to_2019.csv")
county_df = pd.read_csv("resources/county_populations_2000_to_2019.csv")


pop_data = pd.merge(county_df,state_df,
                     on="state_name",
                     how="left")
# displaying result
print(pop_data)

     state_name        county_name  county_FIPS  pop_2000_x  pop_2001_x  \
0       Alabama     Autauga County         1001       43872       44434   
1       Alabama     Baldwin County         1003      141358      144988   
2       Alabama     Barbour County         1005       29035       29223   
3       Alabama        Bibb County         1007       19936       20942   
4       Alabama      Blount County         1009       51181       51999   
...         ...                ...          ...         ...         ...   
3138    Wyoming  Sweetwater County        56037       37484       36566   
3139    Wyoming       Teton County        56039       18367       18594   
3140    Wyoming       Uinta County        56041       19662       19406   
3141    Wyoming    Washakie County        56043        8257        8023   
3142    Wyoming      Weston County        56045        6635        6472   

      pop_2002_x  pop_2003_x  pop_2004_x  pop_2005_x  pop_2006_x  ...  \
0          45157       457

In [513]:
pop_county = pop_data["county_name"]
pop_county.head()

0    Autauga County
1    Baldwin County
2    Barbour County
3       Bibb County
4     Blount County
Name: county_name, dtype: object

In [514]:
state_df.columns

Index(['state_name', 'state_FIPS', 'pop_2000', 'pop_2001', 'pop_2002',
       'pop_2003', 'pop_2004', 'pop_2005', 'pop_2006', 'pop_2007', 'pop_2008',
       'pop_2009', 'pop_2010', 'pop_2011', 'pop_2012', 'pop_2013', 'pop_2014',
       'pop_2015', 'pop_2016', 'pop_2017', 'pop_2018', 'pop_2019'],
      dtype='object')

In [515]:
state_county_df = pop_data[["state_name", "county_name"]]
#sc = state_county_df.groupby (["state_name","county_name"])
state_county_df.nunique()

state_name       51
county_name    1877
dtype: int64

In [516]:
landfill_df = pd.read_csv("resources/openlandfills.csv")
landfill_df.dropna(inplace = True)
landfill_df.head()

,State,Landfill Name,Current Landfill Status,Latitude,Longitude
0,AK,Anchorage Regional Landfill,Open,61.293281,-149.602138
1,AK,Capitol Disposal Landfill,Open,58.352800,-134.494700
2,AK,Central Peninsula Landfill (CPL),Open,60.447140,-151.103690
3,AK,Palmer Central Landfill,Open,61.590000,-149.210000
4,AK,South Cushman Landfill,Open,64.804760,-147.700850


In [517]:
# base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
# response = requests.get(base_url, params)

In [518]:
# Store latitude and longitude in locations
locations = landfill_df[["Latitude", "Longitude"]]

In [519]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [520]:
county_cancer_df = pd.read_csv("resources/eEdited_incd.csv")
county_cancer_df.dropna(inplace = True)
county_cancer_df.head()

,Unnamed: 0,County,State_x,State_y,FIPS,"Age-Adjusted Incidence Rate([rate note]) - cases per 100,000",Average Annual Count,Recent Trend
0,0,Union County,Florida,"Union County, Florida",12125,1143.5,213,stable
1,1,Logan County,Nebraska,"Logan County, Nebraska",31113,739.4,7,*
2,2,Greer County,Oklahoma,"Greer County, Oklahoma",40055,710.4,52,rising
3,3,Powell County,Kentucky,"Powell County, Kentucky",21197,648.2,93,stable
4,4,Sherman County,Nebraska,"Sherman County, Nebraska",31163,629.5,28,stable


In [522]:
# county_clean_df['Cases'] = county_clean_df['Cases'].str.replace("#", "")
# county_clean_df = county_cancer_df.rename(columns={"Age-Adjusted Incidence Rate([rate note]) - cases per 100,000": "Cases", "State_x": "State"})

In [ ]:
county_clean_df.head()

In [523]:
# target = "County"

# params = {"administrative_area_level_2": target}   

# base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# response = requests.get(base_url, params=params)

# # Convert to JSON
# cancer_geo = response.json()

# #Extract lat/lng
# lat = cancer_geo["results"][0]["geometry"]["target"]["lat"]
# lng = cancer_geo["results"][0]["geometry"]["target"]["lng"]

# # #Store county and state 
# # location = county_clean_df[["County","State"]]
# # rating = county_clean_df[["Cases"]].astype(float)

# print(f"{target_city}: {lat}, {lng}")

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
cancer_layer = gmaps.heatmap_layer(location, rating,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(cancer_layer)

# Display figure
fig

In [ ]:
county_longlat = pd.read_csv("resources/cities.csv")
county_cancer_.dropna(inplace = True)
county_cancer_df.head()

In [ ]:
In [ ]:
# load csv
# Combime files
# delete unused columns
# identify state, county
# Merge landfill data, group by state and county
# Identify the highest landfill states
# merge cancer data, overlay on state and conty data
# create a 100 miles radius based on address of landfill
# find similar population and landfill dataa( similar to weather or gmaps homework assignment?)
# If silimair popolations with landfills have similar cancer rates, this may prove a higher risk
# Can we use gmaps to rank the landfills with the highest cancer radius?